In [ ]:
import math
import numpy as np
import torch
from torch.autograd import Variable
import pandas as pd
import scipy.stats as stats

In [ ]:
# 正規化の関数
# def mean_norm(df_input): # 標準化
#         return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
    
def min_max_norm(x, axis=None): # 正規化
    x_min = x.min(axis=axis)
    x_max = x.max(axis=axis)
    result = (x-x_min)/(x_max-x_min)
    return result

In [ ]:
d = 3 # 入力値のサイズ
t = 1 # 出力値のサイズ（次元数）
h = 50 # 隠れ層のノード数
k = 30 # 正規分布の山の数
np.random.seed(seed=32)

## MDN

In [ ]:
import MDN_model
model = MDN_model.MDN_NN(d,t,h,k)
model.load_state_dict(torch.load('save.pth')) # load

In [ ]:
# 正規化から元に戻す関数
def return_norm(tmp_df, base_df, axis=None):
    """
    tmp_df : 修正するデータ
    base_df : 元にするデータ（min, maxをとるデータ）
    """
    x_max = base_df.max()
    x_min = base_df.min()
    return tmp_df.apply(lambda x: x*(x_max-x_min)+x_min)

In [ ]:
# 順番に出力
# stateごとに図を作成

# 元々のデータ
df = MDN_model.get_df_data()
sample_num = 100
x_max = df["PlayResult"].max()
x_min = df["PlayResult"].min()

def min_max_norm(x): # 正規化
    result = (x-x_min)/(x_max-x_min)
    return result

# 出力のための辞書
yard_dict = {1:"red", 2:"yellow", 3:"blue"}
dist_dict = {1:"long", 2:"middle", 3:"short"}
dn_dict = {1:"1st down", 2:"2nd down", 3:"3rd down"}
play_dict = {0:"guard", 1:"tackle", 2:"end", 3:"short", 4:"deep"}

# 出力するdf
l_p  = []

for dn in range(1,4):
    for yard in range(0,11):
        for play in range(0,5):
            particle_list = [dn, 10 - yard, play]
            # テストデータ
            x_test = [particle_list for i in range(1)]
            x_test = torch.from_numpy(np.array(x_test))
            x_test = x_test.double()
            # 順伝搬
            pi, sigmasq, mu = model(Variable(x_test))
            pi      = pi.detach().numpy()[0]
            sigmasq = sigmasq.detach().numpy()[0]
            mu      = mu.detach().numpy()[0]
            ps = np.zeros(11)
            for x in range(11):
                x_ub = min_max_norm(x+1)
                x_lb = min_max_norm(x)
                
                p = 0
                if yard + x < 10:
                    p_ub = 0
                    for k_ in range(k):
                        p_ub += pi[k_] * stats.norm.cdf(x_ub, loc=mu[k_], scale=math.sqrt(sigmasq[k_]))
                else:
                    p_ub = 1
                
                if x != 0:
                    p_lb = 0
                    for k_ in range(k):
                        p_lb += pi[k_] * stats.norm.cdf(x_lb, loc=mu[k_], scale=math.sqrt(sigmasq[k_]))
                else:
                    p_lb = 0
                    
                ps[yard + x] = p_ub - p_lb
                if yard + x == 10:
                    break
            for i in range(11):
                l_p.append([dn,yard,play,i,ps[i]])
                
df_p = pd.DataFrame(l_p,columns=["dn","yard","a","n_yard","p"])
df_p.to_csv("mdn_p.csv")